# rdrp Benchmark / GITB
```
Lead     : ababaian /rce
Issue    : 
start    : 2020 12 10
complete : 2020 12 13
revision : 2020 12 15
files    : ~/serratus/notebook/201210_rdrp0/
s3 files : s3://serratus-public/notebook/201210_rdrp0/
```

## Introduction

> `rce:`
> For validation of RdRp identification and trimming, we should do hold-out cross-validation using high-quality sequences, i.e. RefSeqs. To assess how accuracy varies with identity, we will use cross-validation by identity (CVI, https://www.drive5.com/usearch/manual/cvi.html) which constructs test/training sets where the maximum identity (MaxID) of the top test-training hit is fixed to a known value or range of values. Here, I suggest five test/training pairs with MaxID = 90, 75, 50, 30, 15% to get a representative range.
>
> The gold standard reference for the gene identification and trimming benchmark (GITB-gold) will be full-length RdRp-containing ORFs from RefSeqs plus coordinates of the trimmed RdRps. These coordinates will be obtained by aligning the ORFs to Wolf18. These alignments should be good (diamond may be good enough) because the top-hit identities should be at least 90%.
>
> The decoy reference (GITB-decoy) will be all RefSeq ORFs which do NOT contain RdRp. Ideally, these will all be discarded by the protocol. In practice, there will be FPs due to local homology between RdRp and other genes; this surely arises often in viruses due to recombination events. Using the decoy reference will enable us to design better filters and measure the FP rate (ROC curve) of the protocol.

Most of the code for this was developed already in `201210_RdRp_panproteome_v1.ipyn` notebook. This is a self-contained migration of that code / clean-up.

In [1]:
# date and version
date

Wed Dec 30 15:25:39 PST 2020


In [ ]:
# Upload gitb folder (from EC2)
aws s3 sync ./ s3://serratus-public/notebook/201230_gitb/

## Materials

- `wolf18.fa` : Trimmed Set of RdRp True Positives
```
# Header fields:
# rdrp_branch.viral_family.viral_name:accession
```

- `vrs240.fa` : Viral RefSeq Sequences (aa)



- `vrs240_w18.fa` : ALL RefSeq Sequences with a wolf18 match (includes low confidence)

- `vrs240_w18.pro` : Diamond TSV output of RefSeq vs. wolf18

### GITB Sequences

- `gitb.fa` : High-Confidence, Full-Length RdRp+ RefSeq Sequences


- `gitb.rdrp.fa` : RdRp RefSeq sequences
```
# Header fields:
# RefSeq_accession rdrp_start rdrp_end rdrp_len
```
- `gitb.pro` : Diamond output high-confidence subset of `vrs240_w18.pro` 

### Cross Validation Sets

- `cvi/`

- `gitb.all.fa` : same as ../gitb.fa
- `gitb.45.cvi`: Cross-validation set at 45% identity (40-50% sequence matchs)

## WOLF18 RdRp Header Parsing

Sequence data: `ftp://ftp.ncbi.nlm.nih.gov/pub/wolf/_suppl/rnavir18/RNAvirome.S2.afa`
Original file: `201230_gitb/wolf18/gb_rdrp.afa`
Parsed file: `201230_gitb/wolf18.fa`

The headers in this data is pre-processe

### Level 1 - Supergroup / Branches

The RdRp can be broadly classified into 5 branches which will form the lowest level of the hierarchy: `rdrp1`, `rdrp2`, `rdrp3`, `rdrp4`, `rdrp5`.

>Branch 1 consists of leviviruses and their eukaryotic relatives, namely, “mitoviruses,” “narnaviruses,” and “ourmiaviruses” (the latter three terms are placed in quotation marks as our analysis contradicts the current ICTV framework, which classifies mitoviruses and narnaviruses as members of one family, Narnaviridae, and ourmiaviruses as members of a free-floating genus, Ourmiavirus).

> Branch 2 (“picornavirus supergroup”) consists of a large assemblage of +RNA viruses of eukaryotes, in particular, those of the orders Picornavirales and Nidovirales; the families Caliciviridae, Potyviridae, Astroviridae, and Solemoviridae, a lineage of dsRNA viruses that includes partitiviruses and picobirnaviruses; and several other, smaller groups of +RNA and dsRNA viruses.

> Branch 3 consists of a distinct subset of +RNA viruses, including the “alphavirus supergroup” along with the “flavivirus supergroup,” nodaviruses, and tombusviruses; the “statovirus,” “wèivirus,” “yànvirus,” and “zhàovirus” groups; and several additional, smaller groups.

> Branch 4 consists of dsRNA viruses, including cystoviruses, reoviruses, and totiviruses and several additional families.

> Branch 5 consists of −RNA viruses.

Boundary defintions of Branches with relation to RdRp are taken from paper

Based on: [Supplementary Data 4](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6282212/bin/mbo006184203sd4.txt)
Saved as: `rdrp_representative_branches.tree`

### Level 2 - Viral Family

Based on: `https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6282212/bin/mbo006184203sd1.xls`
Saved as: `wolf18/wolf18_vlist.xlsx`

Spreadsheet includes the fields

- RdRp num ID: Ordinal numbering for RdRp
- RdRp GenBank Acc: Protein accession ID
- NCBI Tax ID: taxid from NCBI
- virus name: virus name
- taxonomy: taxonomic tree

Taxonomy field was parsed to retrieve "*dae* suffix for "Family", relatively appropriate family-name or "unclassified" when unavailable. Monkey work.

### Level 3 - Sequence/Species

Based on: `wolf18_vlist.xlsx`

- Virus name field (most shallow taxonomic classification) taken for each record.
- GenBank accession taken from each record.

### Example RdRp

```
rdrp5.Hantaviridae.Bowe_virus:AGW23849.1
rdrp5.Bunyaviridae.Azagny_virus:AEA42011.1
...
rdrp2.Coronaviridae.Night_heron_coronavirus_HKU19:YP_005352862.1
rdrp2.Coronaviridae.Munia_coronavirus_HKU13_3514:YP_002308505.1
rdrp2.Coronaviridae.Wigeon_coronavirus_HKU20:YP_005352870.1
rdrp2.Coronaviridae.Feline_infectious_peritonitis_virus:AGZ84535.1
rdrp2.Coronaviridae.Lucheng_Rn_rat_coronavirus:YP_009336483.1
rdrp2.Coronaviridae.Hipposideros_bat_coronavirus_HKU10:AFU92121.1
rdrp2.Coronaviridae.BtMs_AlphaCoV_GS2013:AIA62270.1
rdrp2.Coronaviridae.Chaerephon_bat_coronavirus_Kenya_KY41_2006:ADX59465.1
rdrp2.Coronaviridae.Porcine_epidemic_diarrhea_virus:AID56804.1
rdrp2.Coronaviridae.Bat_coronavirus_CDPHE15_USA_2006:YP_008439200.1
rdrp2.Coronaviridae.Anlong_Ms_bat_coronavirus:AID16674.1
rdrp2.Coronaviridae.Scotophilus_bat_coronavirus_512:YP_001351683.1
rdrp2.Coronaviridae.BtNv_AlphaCoV_SC2013:YP_009201729.1
rdrp2.Coronaviridae.Bat_coronavirus_1B:ACA52156.1
rdrp2.Coronaviridae.NL63_related_bat_coronavirus:YP_009328933.1
rdrp2.Coronaviridae.NL63_related_bat_coronavirus:APD51489.1
rdrp2.Coronaviridae.229E_related_bat_coronavirus:ALK43115.1
rdrp2.Coronaviridae.Rhinolophus_bat_coronavirus_HKU2:ABQ57223.1
rdrp2.Coronaviridae.Wencheng_Sm_shrew_coronavirus:AID16677.1
rdrp2.Coronaviridae.Human_coronavirus_HKU1:ABD75543.1
rdrp2.Coronaviridae.Betacoronavirus_Erinaceus_VMC_DEU_2012:YP_008719930.1
rdrp2.Coronaviridae.Pipistrellus_bat_coronavirus_HKU5:YP_001039961.1
rdrp2.Coronaviridae.Rousettus_bat_coronavirus:AOG30811.1
rdrp2.Coronaviridae.Rousettus_bat_coronavirus:YP_009273004.1
rdrp2.Coronaviridae.Bat_CoV_279_2005:P0C6V9.1
rdrp2.Coronaviridae.Bat_Hp_betacoronavirus_Zhejiang2013:YP_009072438.1
rdrp2.Coronaviridae.Bottlenose_dolphin_coronavirus_HKU22:AHB63494.1
rdrp2.Coronaviridae.Duck_coronavirus:AKF17722.1
rdrp2.Coronaviridae.Avian_infectious_bronchitis_virus_partridge_GD_S14_2003:AAT70770.1
rdrp2.Coronaviridae.Infectious_bronchitis_virus:ADA83556.1
...
rdrp1.unclassified.Wenzhou_levi_like_virus_3:APG77299.1
rdrp1.Leviviridae.Pseudomonas_phage_PP7:NP_042307.1

```

saved as: `wolf18/gb_assign_group_v2.txt`

In [ ]:
# Parsing Code

# Rename the header to remove virus name
# remove gaps from sequence (unaligned)
sed 's/|.*//g' gb_rdrp.afa \
  | sed 's/-//g' - \
  > rdrp_1.tmp

# One sequence "Pseudomonas_phage_phiYY" has no accession
# YP_009618381.1
sed -i 's/^>$/>YP_009618381.1/g' rdrp_1.tmp

# Iterate through each line / fasta name.
# to swap out headers
while read -r line; do
  # Find headers
  if [[ "$line" = ">"* ]]; then
    acc=$(echo $line | sed 's/\..*//g' - | sed 's/>//g' -)
    newheader=$(grep "$acc" gb_assign_group_v2.txt)
    
    if [[ "$newheader" = "" ]]; then
      echo ">NA_$acc" >> rdrp_2.tmp
    else
      echo $newheader >> rdrp_2.tmp
    fi
    
  else
    # print aa sequence
    echo $line >> rdrp_2.tmp 
  fi
done < rdrp_1.tmp


# Manually remove first ten sequences (Group II introns outgroup)
tail -n +21 rdrp_2.tmp > rdrp_3.tmp

mv rdrp_3.tmp ../wolf18.fa

#rm *.tmp

## Viral RefSeq 240


In [ ]:
# RefSeq (rs) folder
mkdir -p rs; cd rs

wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.protein.faa.gz
wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.2.protein.faa.gz
gzip -d *.gz

cat viral.1.protein.faa viral.2.protein.faa > vrs240.fa
rm viral*faa

In [ ]:
## Run Diamond with ULTRA-SENSITIVE mode
QUERY='vrs240.fa'
RDRP='wolf18'
OUTNAME='vrs240_w18'

diamond makedb --in $RDRP.fa -d $RDRP

# Diamond blastp alignment
time cat $QUERY |\
diamond blastp \
  -d "$RDRP".dmnd \
  --unal 0 \
  --masking 0 \
  --ultra-sensitive \
  -k 1 \
  -p 16 \
  -b 1 \
  -f 6 qseqid qstart qend qlen qstrand \
       sseqid sstart send slen \
       pident evalue cigar \
       qseq \
  > "$OUTNAME".pro


# Fasta of local diamond matches
cut -f1,2,3,4,6,7,8,9,10,13 $OUTNAME.pro \
  | sed 's/^/>/g' \
  | sed 's/\(\t\)\([A-Z]*$\)/\n\2/g' - \
  > $OUTNAME.fa

In [ ]:
# Extract High confidence set of matches
GITB='gitb'

# Extract only matches that are:
# >88% identity to rdrp0
# >98% coverage of rdrp0

cut -f1,2,3,4,6,7,8,9,10,13 $OUTNAME.pro \
  > pro.tmp

rm $GITB.pro

while read -r line; do
  
  pctid=$(echo $line | cut -f 9 -d' ' - )  
  
  subj_start=$(echo $line | cut -f 6 -d' ' - )
  subj_end=$(echo $line | cut -f 7 -d' ' - )
  subj_len=$(echo $line | cut -f 8 -d' ' - )
  
  subj_cov=$( echo "$subj_end - $subj_start + 1" | bc )
  subj_covpct=$( echo "scale=2;$subj_cov / $subj_len" | bc )
  
  if [[ $pctid > 0.88 ]] && [[ $subj_covpct > 0.98 ]]; then
    echo $line >> $GITB.pro
  fi
  
done < pro.tmp
rm pro.tmp


In [ ]:
# Parse GITB set into fasta file
# Fields:
# Query_accession rdrp_start rdrp_end rdrp_len
cut -f1,2,3,10 -d' ' $GITB.pro \
  | sed 's/^/>/g' \
  | sed 's/\( \)\([A-Z]*$\)/\n\2/g' - \
  > $GITB.rdrp.fa

# rdrp sequences in GITB as bed3
cut -f1,2,3 -d' ' $GITB.pro \
  | sed 's/ /\t/g' - \
  > $GITB.rdrp.bed

# From GITB, retrieve full-length records from RefSeq
# (complete ORFs)
cut -f1 $GITB.rdrp.bed | sed 's/>//g' - > $GITB.hits
seqkit grep -r -f $GITB.hits vrs240.fa > $GITB.fa


## Cross-Validation Set of GITB

In [ ]:
# Create Cross-Validation Set
mkdir cvi; cd cvi
GITB="gitb"

# High-Confidence RDRP only
ln -s ../gitb.fa ./$GITB.all.fa

# USEARCH CVI
usearch -threads 30 -calc_distmx $GITB.all.fa -maxdist 1 -termdist 1 -tabbedout gitb.distmat

# 90% CVI 
usearch -distmx_split_identity  gitb.distmat -mindist 0.05 -maxdist 0.15 \
  -tabbedout $GITB.90.cvi

# 75% CVI 
usearch -distmx_split_identity  gitb.distmat -mindist 0.2 -maxdist 0.3 \
  -tabbedout $GITB.75.cvi

# 65% CVI 
usearch -distmx_split_identity  gitb.distmat -mindist 0.3 -maxdist 0.4 \
  -tabbedout $GITB.65.cvi

# 55% CVI 
usearch -distmx_split_identity  gitb.distmat -mindist 0.4 -maxdist 0.5 \
  -tabbedout $GITB.55.cvi

# 45% CVI 
usearch -distmx_split_identity  gitb.distmat -mindist 0.5 -maxdist 0.6 \
  -tabbedout $GITB.45.cvi

# 25% CVI 
usearch -distmx_split_identity gitb.distmat -mindist 0.7 -maxdist 0.8 \
  -tabbedout $GITB.25.cvi
  
#1 Subset name.
#2 Label1
#3 Label2
#4 Dist